In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
app_data=pd.read_csv("googleplaystore.csv")
app_review=pd.read_csv("googleplaystore_user_reviews.csv")

In [3]:
app_data.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [4]:
app_data["Installs"]=app_data["Installs"].str.replace(",","",regex=True)
app_data["Installs"]=app_data["Installs"].str.replace(".$","",regex=True)

In [5]:
app_data=app_data[app_data["Installs"]!="Fre"]
app_data=app_data[app_data["Installs"]!=""]
app_data["Installs"]=pd.to_numeric(app_data["Installs"])

In [6]:
app_data=app_data[app_data["Installs"]>10000]
app_data.drop(columns="Installs",inplace=True)

In [7]:
app_data["Reviews"]=pd.to_numeric(app_data["Reviews"])
app_data=app_data[app_data["Reviews"]>1000]

In [8]:
app_data.drop(columns="Reviews",inplace=True)

In [9]:
app_data["Content Rating"].value_counts()
mask1=app_data["Content Rating"].isin(["Everyone","Teens"])
app_data=app_data[mask1]

mask2=app_data["Type"]=="Free"
app_data=app_data[mask2]

In [10]:
app_data.head()

,App,Category,Rating,Size,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,8.7M,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
7,Infinite Painter,ART_AND_DESIGN,4.1,29M,Free,0,Everyone,Art & Design,"June 14, 2018",6.1.61.1,4.2 and up
8,Garden Coloring Book,ART_AND_DESIGN,4.4,33M,Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up
10,Text on Photo - Fonteee,ART_AND_DESIGN,4.4,28M,Free,0,Everyone,Art & Design,"October 27, 2017",1.0.4,4.1 and up
11,Name Art Photo Editor - Focus n Filters,ART_AND_DESIGN,4.4,12M,Free,0,Everyone,Art & Design,"July 31, 2018",1.0.15,4.0 and up


In [11]:
app_data.shape

(4205, 11)

In [12]:
app_data.drop(columns=["Size","Type","Content Rating","Genres"],inplace=True)

In [13]:
x=app_data["Category"].value_counts()>50
x=x[x]
listx=x.index.tolist()

In [14]:
app_data.replace("Varies with device",np.NaN,inplace=True)

In [15]:
app_data["Current Ver"]=pd.to_numeric(app_data["Current Ver"],errors="coerce")

In [16]:
app_data["Android Ver"]=app_data["Android Ver"].str.replace(" and up","",regex=True)
app_data=app_data[~app_data["Android Ver"].isin(["4.0.3 - 7.1.1","5.0 - 8.0","4.1 - 7.1.1"])]
app_data["Android Ver"].replace({"4.0.3":"4.0","2.3.3":"2.3","2.0.1":"2.0"},inplace=True)
app_data["Android Ver"].fillna(4.0,inplace=True)
app_data["Current Ver"].fillna(app_data["Current Ver"].mean(),inplace=True)


In [17]:
app_data.dropna(inplace=True)
app_data.shape

(4195, 7)

In [18]:
app_review.shape

(64295, 5)

In [19]:
app_review.drop(columns=["Sentiment_Polarity","Sentiment_Subjectivity"],inplace=True)

In [20]:
app_review.dropna(inplace=True)
app_review.shape

(37427, 3)

In [21]:
app_data=app_data.merge(app_review,on="App")

In [22]:
app_review.columns

Index(['App', 'Translated_Review', 'Sentiment'], dtype='object')

In [23]:
app_data["Price"]=pd.to_numeric(app_data["Price"].str.replace("$","",regex=True))

In [24]:
app_data.shape

(51946, 9)

In [25]:
app_data.drop(columns="App",inplace=True)
app_data.columns

Index(['Category', 'Rating', 'Price', 'Last Updated', 'Current Ver',
       'Android Ver', 'Translated_Review', 'Sentiment'],
      dtype='object')

In [26]:
app_data.drop(columns="Category",inplace=True)

In [27]:
app_data.drop(columns="Last Updated",inplace=True)
app_data.columns

Index(['Rating', 'Price', 'Current Ver', 'Android Ver', 'Translated_Review',
       'Sentiment'],
      dtype='object')

In [28]:
app_data["Android Ver"]=pd.to_numeric(app_data["Android Ver"])

In [29]:
y=app_data.iloc[:,0].values

In [30]:
words=[]
for i in app_data.iloc[:,4]:
    words+=i.split()

In [31]:
len(words)

1115408

In [32]:
preplist=["about","above","across","after","against","among","around","at","before","behind","below","beside","between","by","down","during","except","for","from","in","inside","into","near","of","off","on","out","over","through","to","toward","under","up","with"]
prepcent=[i.capitalize() for i in preplist]
prepupper=[i.upper() for i in preplist]

pronouns=["I","we","you","he","she","it","they","me","us","you","her","him","it","them"]
procent=[i.capitalize() for i in pronouns]
proupper=[i.upper() for i in pronouns]

conjunctions=["for", "and", "nor", "but", "or", "yet", "so", "after", "although", "as", "because", "before", "if", "than", "that", "though", "till", "unless", "until", "when", "whenever", "where", "wherever", "while", "neither", "nor", "either"]
concent=[i.capitalize() for i in conjunctions]
conupper=[i.upper() for i in conjunctions]

determiners=["the","a","an","this", "that", "these", "those", "my", "your", "his", "her", "its", "our", "their", "much", "many", "most", "some", "any", "enough","all", "both", "half", "either", "neither", "each", "every", "other", "another", "such", "what", "rather", "quite"]
detcent=[i.capitalize() for i in determiners]
detupper=[i.upper() for i in determiners]

for i in range(len(words)):
    if not words[i].isalpha():
        words[i]=""
    if ((words[i] in preplist) or (words[i] in prepcent) or(words[i] in prepupper)):
        words[i]=""
    if ((words[i] in pronouns) or (words[i] in procent) or (words[i] in proupper)):
        words[i]=""
    if ((words[i] in conjunctions) or (words[i] in concent) or (words[i] in conupper)):
        words[i]=""
    if ((words[i] in determiners) or (words[i] in detcent) or (words[i] in detupper)):
        words[i]=""


In [33]:
len(words)

1115408

In [34]:
from collections import Counter

In [35]:
word_dict=Counter(words)
del word_dict[""]
len(word_dict)

18194

In [36]:
word_dict=word_dict.most_common(2000)

In [37]:
len(word_dict)

2000

In [38]:
features=[]

In [39]:
for i in app_data.iloc[:,4]:
    blob=i.split()
    data=[]
    
    for j in word_dict:
        data.append(blob.count(j[0]))
    features.append(data)

In [40]:
len(features)

51946

In [41]:
app_data.columns

Index(['Rating', 'Price', 'Current Ver', 'Android Ver', 'Translated_Review',
       'Sentiment'],
      dtype='object')

In [42]:
from sklearn.preprocessing import LabelEncoder

In [43]:
encoder=LabelEncoder()

In [44]:
app_data["Sentiment"]=encoder.fit_transform(app_data.iloc[:,2].values)

In [45]:
from sklearn.preprocessing import OneHotEncoder

In [46]:
oencoder=OneHotEncoder(categorical_features=[0])
labelled=oencoder.fit_transform(app_data.iloc[:,5:].values)

/usr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [47]:
labelled=labelled.toarray()
labelled=labelled[1:-1]
labelled.shape

(51944, 72)

In [48]:
features=np.array(features)
app_data.drop(columns="Translated_Review",inplace=True)
app_data.columns

Index(['Rating', 'Price', 'Current Ver', 'Android Ver', 'Sentiment'], dtype='object')

In [49]:
app_data.drop(columns="Sentiment",inplace=True)

In [50]:
app_data.iloc[:,1:4].columns

Index(['Price', 'Current Ver', 'Android Ver'], dtype='object')

In [51]:
features=np.append(features,app_data.iloc[:,1:4].values,axis=1)
app_data.iloc[:,1:4].values.shape

(51946, 3)

In [52]:
features.shape

(51946, 2003)

In [53]:
from sklearn.model_selection import train_test_split

In [54]:
X_train,X_test,y_train,y_test=train_test_split(features,y,test_size=0.2,random_state=9)

In [55]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [56]:
from sklearn.ensemble import RandomForestRegressor

In [57]:
regressor=RandomForestRegressor(n_estimators=10,verbose=3,n_jobs=-1)
regressor.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 10building tree 2 of 10building tree 3 of 10building tree 4 of 10



building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  8.2min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 10.6min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=3, warm_start=False)

In [58]:
y_pred=regressor.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.1s finished


In [59]:
from sklearn.metrics import r2_score

In [60]:
r2_score(y_test,y_pred)

0.6651681012013817

In [61]:
y_test.shape

(10390,)

In [65]:
y_pred.shape

(10020,)